<a href="https://colab.research.google.com/github/mitazzdev/predict_thainumber/blob/main/brain_tumors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive

os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive'

# Construct the full file path
file_path = "/content/Data"

# Check if the file exists
if os.path.exists(file_path):
    print(f'The file Data exists in the directory.')
else:

    drive.mount('/content/drive')

    # Download the Kaggle dataset
    !kaggle datasets download -d thomasdubail/brain-tumors-256x256

    # Unzip the downloaded dataset
    !unzip brain-tumors-256x256.zip


The file Data exists in the directory.


In [ ]:
model_name = '1'
epochs = 20  # Adjust the number of training epochs
learning_rate= 0.01
drop_out = 0.7

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define your data directory
data_dir = '/content/Data'

# Define parameters for data preprocessing
image_size = (128, 128)  # Set your desired image size
batch_size = 32

# Load and preprocess the data
datagen = ImageDataGenerator(
    rescale=1./255,          # Normalize pixel values to [0, 1]
    rotation_range=20,       # Data augmentation: random rotation up to 20 degrees
    width_shift_range=0.1,  # Data augmentation: random horizontal shift
    height_shift_range=0.1, # Data augmentation: random vertical shift
    horizontal_flip=True,    # Data augmentation: random horizontal flip
    validation_split=0.2    # Split the data into training and validation
)

# Load and split the data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Change this depending on your task
    subset='training',         # Specify training data
    seed=42
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Change this depending on your task
    subset='validation',       # Specify validation data
    seed=42
)

# Split the data into training, validation, and test sets
# You can also use scikit-learn's train_test_split for this
X_train, X_test, y_train, y_test = train_test_split(
    train_generator[0][0], train_generator[0][1], test_size=0.2, random_state=42)

# Optionally, you can create a test data generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',  # Change this depending on your task
    shuffle=False,             # No need to shuffle test data
)

# Now you have training, validation, and test data ready for your CNN model.


Found 2479 images belonging to 4 classes.
Found 617 images belonging to 4 classes.
Found 3096 images belonging to 4 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

num_classes = 4

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))  # Adjust input_shape as per your data
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(drop_out))  # Optional dropout for regularization
model.add(Dense(num_classes, activation='softmax'))  # Adjust num_classes for your problem

# Compile the model
model.compile(optimizer=Adam(learning_rate),
              loss='categorical_crossentropy',  # Adjust loss function as per your task
              metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train,  # Your training data
    y_train,  # Your training labels
    epochs=epochs,
    validation_data=(validation_generator[0][0], validation_generator[0][1]),  # Validation data
)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(
    test_generator[0][0], test_generator[0][1], verbose=2)

print("Test accuracy:", test_accuracy)

# Make predictions on new data
# predictions = model.predict(new_data)  # Replace 'new_data' with your data

# Save the model
model.save(f'/content/drive/MyDrive/Model/{model_name}.keras')


1/1 [==============================] - 1s 868ms/step - loss: 1.5240 - accuracy: 0.2400 - val_loss: 1.3223 - val_accuracy: 0.3438
Epoch 6/20
1/1 [==============================] - 1s 798ms/step - loss: 1.3621 - accuracy: 0.2000 - val_loss: 1.3470 - val_accuracy: 0.3438
Epoch 7/20
1/1 [==============================] - 1s 1s/step - loss: 1.2745 - accuracy: 0.5200 - val_loss: 1.3025 - val_accuracy: 0.3750
Epoch 8/20
1/1 [==============================] - 2s 2s/step - loss: 1.2089 - accuracy: 0.4800 - val_loss: 1.2745 - val_accuracy: 0.2812
Epoch 9/20
1/1 [==============================] - 1s 1s/step - loss: 1.3064 - accuracy: 0.5200 - val_loss: 1.3076 - val_accuracy: 0.2188
Epoch 10/20
1/1 [==============================] - 1s 836ms/step - loss: 1.1545 - accuracy: 0.7200 - val_loss: 1.2944 - val_accuracy: 0.3438
Epoch 11/20
1/1 [==============================] - 1s 851ms/step - loss: 1.1593 - accuracy: 0.4800 - val_loss: 1.2619 - val_accuracy: 0.3438
Epoch 12/20
1/1 [=====================